<a href="https://colab.research.google.com/github/gracieleo/imersao-ia-alura-google/blob/master/Aula_05_Imers%C3%A3o_IA_Alura_e_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Aula 5: Criando um sistema para busca em documentos usando embeddings e a Gemini API

In [1]:
!pip install -q -U google-generativeai

In [2]:
#importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=API_KEY)

In [3]:
#listar modelos embedding do Gemini
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
#exemplo de embedding
title = "O que são embeddings?"
sample_text = ("Embeddings de texto são uma técnica de processamento de linguagem natural (PLN) que converte"
               "\n"
               "texto em vetores numéricos. Os embeddings capturam significado semântico e contexto, resultando"
               "\n"
               "em um texto com significados semelhantes com embeddings mais próximos.")

embeddings = genai.embed_content(model="models/embedding-001",
                                content=sample_text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [-0.03916531, -0.019239627, 0.0021091714, 0.07041565, 0.018408595, 0.030073028, 0.009232085, 0.025853774, 0.0053570536, 0.03176475, -0.019521095, -0.011279218, -0.011476373, -0.025971496, -0.019357083, -0.03714979, 0.00065773586, 0.030873705, 0.026218696, -0.03865276, -0.026904877, 0.011300457, 0.01784563, -0.032639228, 0.0048207333, -0.021502102, 0.0155554, -0.0317668, -0.019028591, 0.0334006, -0.030427532, 0.051135287, -0.01539728, 0.005083855, 0.0056268685, 0.0017078386, 0.008247795, -0.05789104, 0.008403304, 0.0056968657, 0.016782956, -0.035868984, -0.006965564, -0.02348171, -0.010540601, -0.009757582, 0.018168263, 0.004796115, -0.017047208, -0.09456071, -0.013916696, -0.04061607, 0.091549955, 0.001628126, -0.047479462, -0.039055206, 0.03679302, -0.026509287, -0.054003865, -0.015828755, -0.00048554665, -0.030459398, 0.008026775, 0.010319701, -0.026236355, 0.004773286, -0.026577756, 0.018257461, 0.037545133, -0.027716903, 0.017068867, -0.025868906, 0.029136155, -0.0001

### Exemplo de treinamento do seu modelo

Cararcteristicas:
- ambiente controlado
- você sabe o que tem nos documentos
- não vai ocorrer alucinações ou desvios significativos
- respostas menos criativas

In [6]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central.  Temperatura: O botão de temperatura controla a temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir a temperatura. Fluxo de ar: O botão de fluxo de ar controla a quantidade de fluxo de ar dentro do carro. Gire o botão no sentido horário para aumentar o fluxo de ar ou no sentido anti-horário para diminuir o fluxo de ar. Velocidade do ventilador: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite que você selecione o modo desejado. Os modos disponíveis são: Auto: O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Cool (Frio): O carro soprará ar frio para dentro do carro. Heat: O carro soprará ar quente para dentro do carro. Defrost (Descongelamento): O carro soprará ar quente no para-brisa para descongelá-lo."}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e controle climático. Para usar a tela sensível ao toque, basta tocar no ícone desejado.  Por exemplo, você pode tocar no ícone \"Navigation\" (Navegação) para obter direções para o seu destino ou tocar no ícone \"Music\" (Música) para reproduzir suas músicas favoritas."}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.  Park (Estacionar): Essa posição é usada quando você está estacionado. As rodas são travadas e o carro não pode se mover. Marcha à ré: Essa posição é usada para dar ré. Neutro: Essa posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está em marcha e não se moverá a menos que você pressione o pedal do acelerador. Drive (Dirigir): Essa posição é usada para dirigir para frente. Low: essa posição é usada para dirigir na neve ou em outras condições escorregadias."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [7]:
#criação de dataframe (transforma, converte a busca acima em formato tabela)

df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


### Fazendo o embedding da lista de documentos

- gera embedding para cada linha (cada documento)
- add uma coluna guardando esse valor de embedding

In [10]:
model = "models/embedding-001"

In [11]:
#função que va gerar o embedding para os documentos 1,2,3
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [12]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.011010795, -0.026731547, -0.036728486, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.015715627, -0.040427547, 0.011117627, 0.002..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.009390755, -0.022475218, -0.0024638234, 0...."


### Gerar consultas

- consultas como fazemos nos prompts, em relação a algo do documento
- as consultas também devem estar em formato embedding
- o produto escalar (distancia) vai ser calculado embedding pergunta x cada embedding documento

In [13]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [14]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.  Park (Estacionar): Essa posição é usada quando você está estacionado. As rodas são travadas e o carro não pode se mover. Marcha à ré: Essa posição é usada para dar ré. Neutro: Essa posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está em marcha e não se moverá a menos que você pressione o pedal do acelerador. Drive (Dirigir): Essa posição é usada para dirigir para frente. Low: essa posição é usada para dirigir na neve ou em outras condições escorregadias.


### Mesclando a interação do modelo embedding acima com o Gemini

- respostas mais criativas

In [19]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [20]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Guia do Googlecar para Descolados**

Seu Googlecar é tipo um carro automático maneiro. Pra trocar as marchas, é só dar um toque na alavanca e escolher o que você quer:

* **Estacionar:** Quando você tá parado e quer que o carro fique quietinho. As rodas ficam travadas e ele não mexe nem um centímetro.
* **Ré:** Pra dar aquela ré maneirinha.
* **Neutro:** Quando você tá parado no farol ou no trânsito. O carro tá desligado e não vai se mexer até você pisar no acelerador.
* **Dirigir:** Pra acelerar pra frente.
* **Baixa:** Pra quando você tá dirigindo na neve ou em lugares escorregadios.
